In [9]:
import random
import math as m
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt


from numpy.core.fromnumeric import mean


print("\\begin{table}[t]")
print("\\begin{center}")
print("\\begin{tabular}{| c | c | c | c | c |}")
print("\hline")
print("Policy & ASC & AHC & AOC & ATC  \\\ \hline")

ss = [20,20,20,20,40,40,40,60,60]
SS = [40,60,80,100,60,80,100,80,100]

def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)

for asdasd in range(9):

  """s es el numero de inventario minimo
  S es el numero de inventario maximo
  I es el nivel de inventario en un momento t
  K es el valor del costo de "envio" 
  i es el valor de costo incremental por item ordenado
  el tiempo requerido para que llegue el delivery desp de una orden de compra es
  una variable random distribuida entre 0.5 y 1 mes
  los tiempos entre demandas son distribuidos exponencialmente
  el tamanipo de la demanda debe ser discreta
  dudas
  que mierda es el numero de policies"""

  nivel_inv = []##lo hacemos array o no?
  s = ss[asdasd] #nivel minimo de inventario
  S = SS[asdasd] #nivel maximo de inventario
  num_meses = 120##numero de meses de la simulacion
  n_pol = 0 #no se (cantidad de meses?)
  t_ult_eve = 0.0
  t_prox_eve = [] #tiene el tiempo de todos los eventos
  cant_pedido = 0 #lo asignamos aca o en otro lugar?
  holding_cost = 1 #costo por mes de item que esta en el inventario
  shortage_cost = 5 #costo de ventas que exceden el inventario, verificar si esta bien la definicion
  setup_cost = 32
  i_cost = 3
  area_holding = 0
  area_shortage = 0
  avg_ordering_cost = 0
  avg_holding_cost = 0 
  avg_shortage_cost = 0
  avg_total_cost = 0
  costo_total = 0.0
  costo_tot_ord = []
  arr_eve = []
  pedido_esperado = False
  minlag = 0.5
  maxlag = 1
  time_next_event = [0,0,0,0]
  time = 0.0
  mean_interdemand = 0.1
  time_next_event[0] = num_meses + 1 ##order_arrival
  time_next_event[1] = time - np.log(random.random())*mean_interdemand ##demand
  time_next_event[2] = num_meses ##report
  time_next_event[3] = 0.0 ##evaluate




  def calcula_tipo_eve(a):
      k = 0
      if(a>=0 and a<0.167):
          k=1
      elif(a>=0.167 and a<0.5):
          k=2
      elif(a>=0.5 and a<0.833):
          k=3
      else:
          k = 4
      return k


  def inicializar():
      global t_prox_eve
      global nivel_inv
      global t_ult_eve
      global costo_tot_ord
      t_prox_eve.append(0.0)
      nivel_inv.append(60)
      t_ult_eve = 0.0
      costo_tot_ord.append(0.0)




  def inventario():
      inicializar()
      global num_meses
      global arr_eve
      global time
      global time_next_event
      n_pol = 12
      costo_tot_ord = 0
      #for x in range(15): #pag 95
      while(time < num_meses):
          t_prox_eve.append(time)
          if(time_next_event[0] < time_next_event[1] and time_next_event[0] < time_next_event[3]):
              time = time_next_event[0]
              order_arrival()
              arr_eve.append(0)
          elif(time_next_event[1] < time_next_event[0] and time_next_event[1] < time_next_event[3]):
              time = time_next_event[1]
              demand()
              arr_eve.append(1)
          #elif(time_next_event[3] < time_next_event[1] and time_next_event[3] < time_next_event[0]):
          else:
              time = time_next_event[3]
              evaluate()
              arr_eve.append(3)
          
      

      """
          pe = prox_evento()
          t_prox_eve.append(t_prox_eve[len(t_prox_eve)-1] + pe)
          if (len(t_prox_eve)==2): ##verificar si esto esta bien
              tipo_prox_eve=2 
          else:
              tipo_prox_eve = calcula_tipo_eve(random.random()) 
          if(tipo_prox_eve==1):
              order_arrival()
          elif(tipo_prox_eve==2):
              demand()
          elif(tipo_prox_eve==3):
              evaluate()
          arr_eve.append(tipo_prox_eve)
      """
      
      time = time_next_event[2]
      report()

  def order_arrival():
      global nivel_inv
      global cant_pedido
      global pedido_esperado
      global time_next_event
      global num_meses
      if nivel_inv[len(nivel_inv)-1]<s and pedido_esperado:
          nivel_inv.append(nivel_inv[len(nivel_inv)-1] + cant_pedido)
          pedido_esperado = False
          time_next_event[0] = num_meses + 1
      else: 
          nivel_inv.append(nivel_inv[len(nivel_inv) - 1])
      
          


  def demand():
      global time
      global time_next_event
      global mean_interdemand
      k = random.random()
      if(k>=0 and k<0.25):
          k=1
      elif(k>=0.25 and k<0.5):
          k=2
      elif(k>=0.5 and k<0.75):
          k=3
      elif(k>=0.75 and k<1):
          k=4
      nivel_inv.append(nivel_inv[len(nivel_inv)-1] - k)
      time_next_event[1] = time - np.log(random.random())*mean_interdemand
      
  def evaluate():
      global S
      global s
      global setup_cost
      global cant_pedido
      global pedido_esperado
      global costo_tot_ord
      global costo_total
      global i_cost
      global time
      global time_next_event
      global minlag
      global maxlag
      if nivel_inv[len(nivel_inv)-1]<s and pedido_esperado == False:
          cant_pedido = S - nivel_inv[len(nivel_inv)-1]
          costo_total = costo_total + setup_cost + i_cost * cant_pedido 
          pedido_esperado = True
          costo_tot_ord.append(costo_tot_ord[len(costo_tot_ord)-1] + costo_total)
          time_next_event[0] = time + random.uniform(minlag,maxlag)
      nivel_inv.append(nivel_inv[len(nivel_inv) - 1])
      time_next_event[3] = time + 1.0


  def report():
      global num_meses
      global avg_ordering_cost
      global avg_holding_cost     
      global avg_shortage_cost 
      global costo_total
      global area_shortage
      global area_holding
      global holding_cost
      global shortage_cost
      global t_prox_eve
      global nivel_inv
      global avg_total_cost
      i = 0
      while (i<len(nivel_inv)-1):
          if nivel_inv[i]<0:
              area_shortage = area_shortage - (t_prox_eve[i+1] - t_prox_eve[i])*nivel_inv[i]
          elif nivel_inv[i]>=0:
              area_holding = area_holding + (t_prox_eve[i+1] - t_prox_eve[i])*nivel_inv[i]
          i = i + 1 
      avg_ordering_cost = costo_total / num_meses
      avg_holding_cost = holding_cost * area_holding / num_meses
      avg_shortage_cost = shortage_cost * area_shortage / num_meses
      avg_total_cost = (costo_total + setup_cost * num_meses) / num_meses


  inventario()
  print(str(s) + ", " + str(S) + " & " + str(truncate(avg_total_cost,2)) + " & " + str(truncate(avg_ordering_cost,2)) + " & " + str(truncate(avg_holding_cost,2)) + " & " + str(truncate(avg_shortage_cost,2)) + " \\\ \hline")




"""
for i in range(len(arr_eve)):
    print(str(arr_eve[i]) + " : " + str(nivel_inv[i + 1]))


print("Inicio   : " + str(nivel_inv[0]))
for i in range(2):
    if(arr_eve[i] == 0):
        print("Arribo   : " + str(nivel_inv[i + 1]) + " time: " + str(t_prox_eve[i]))
    if(arr_eve[i] == 1):
        print("Demanda  : " + str(nivel_inv[i + 1]) + " time: " + str(t_prox_eve[i]))
    if(arr_eve[i] == 3):
        print("Pedido   : " + str(nivel_inv[i + 1]) + " time: " +  str(t_prox_eve[i]))

"""


print("\\end{tabular}")
print("\\caption{Inventario}")
print("\\label{tab:fruta}")
print("\\end{center}")
print("\\end{table}")

"""
print(avg_shortage_cost)
print(avg_holding_cost)
print(avg_ordering_cost)
print(avg_total_cost)

sum= 0
for i in range(len(nivel_inv)):
    sum = sum + nivel_inv[i]

print(sum/len(nivel_inv))
"""

\begin{table}[t]
\begin{center}
\begin{tabular}{| c | c | c | c | c |}
\hline
Policy & ASC & AHC & AOC & ATC  \\ \hline
20, 40 & 128.78 & 96.78 & 9.75 & 17.19 \\ \hline
20, 60 & 121.15 & 89.15 & 17.05 & 16.71 \\ \hline
20, 80 & 121.1 & 89.1 & 24.5 & 10.68 \\ \hline
20, 100 & 116.5 & 84.5 & 35.99 & 8.26 \\ \hline
40, 60 & 128.85 & 96.85 & 27.68 & 1.25 \\ \hline
40, 80 & 126.2 & 94.2 & 34.0 & 2.39 \\ \hline
40, 100 & 119.3 & 87.3 & 44.4 & 1.49 \\ \hline
60, 80 & 135.4 & 103.4 & 45.23 & 0.06 \\ \hline
60, 100 & 123.95 & 91.95 & 54.66 & 0.08 \\ \hline
\end{tabular}
\caption{Inventario}
\label{tab:fruta}
\end{center}
\end{table}


'\nprint(avg_shortage_cost)\nprint(avg_holding_cost)\nprint(avg_ordering_cost)\nprint(avg_total_cost)\n\nsum= 0\nfor i in range(len(nivel_inv)):\n    sum = sum + nivel_inv[i]\n\nprint(sum/len(nivel_inv))\n'